In [1]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder
from transformers import AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle
import re

In [2]:
valid_dataset = load_from_disk("/opt/ml/data/train_dataset/validation")

In [3]:
with open('/opt/ml/data/elastic_valid_100.bin','rb') as f:
    elastic_valid = pickle.load(f)
with open('/opt/ml/data/dense_valid_retrieval.bin','rb') as f:
    dense_valid = pickle.load(f)

In [4]:
query = valid_dataset['question']
context = valid_dataset['context']

In [5]:
def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\\n', ' ', text) # remove newline character
    text = re.sub(r'\s+', ' ', text) # remove continuous spaces
    text = re.sub(r'#', ' ', text)
    return text

In [6]:

top_k_list = [1 ,5, 10, 20, 50]

for top_k in top_k_list:
    tfdif_acc = 0
    dense_acc = 0
    for i in range(len(query)):
        q = query[i]
        ground_truth = preprocess(context[i])
        if ground_truth in elastic_valid[q][:top_k]:
            tfdif_acc += 1
        if ground_truth in dense_valid[q][:top_k]:
            dense_acc += 1
    print('score_top_k : ', top_k)
    print('elastic ACC : ', tfdif_acc / len(query))
    print('Dense ACC : ', dense_acc / len(query))
    print()


score_top_k :  1
elastic ACC :  0.7166666666666667
Dense ACC :  0.2708333333333333

score_top_k :  5
elastic ACC :  0.8583333333333333
Dense ACC :  0.43333333333333335

score_top_k :  10
elastic ACC :  0.9
Dense ACC :  0.5166666666666667

score_top_k :  20
elastic ACC :  0.9375
Dense ACC :  0.6

score_top_k :  50
elastic ACC :  0.9541666666666667
Dense ACC :  0.7041666666666667

